This Juypter Notebooked titled 'Exploring_Riot_API' isn’t endorsed by Riot Games and doesn’t reflect the views or opinions of Riot Games
or anyone officially involved in producing or managing League of Legends. League of Legends and Riot Games are
trademarks or registered trademarks of Riot Games, Inc. League of Legends © Riot Games, Inc.

# Data Wrangling and Cleaning

In [1]:
%matplotlib inline
import pandas as pd
import requests
import time
import re
import pickle

## Obtaining Data from API

In [11]:
#ENTER API KEY HERE as string
#go to https://developer.riotgames.com/ to obtain API KEY

#TEMP KEY
api_key = 'RGAPI-3705e379-b31b-4406-a2f2-06536cf55375'

In [12]:
def get_acct_id(summoner_name, region, key):
    """
    Returns the encrypted summonerID for a players in game summoner name.
    
    summoner_name: string, in game summoner name
    region: string
    key: string
    """
    url = 'https://' + region +'.api.riotgames.com/lol/summoner/v4/summoners/' +\
          'by-name/' + summoner_name + '?api_key=' + key
    response = requests.get(url)
    
    if response.status_code == 200:
        print('App Rate Limit ' + response.headers['X-App-Rate-Limit'])
        print('App Rate Limit Count ' + response.headers['X-App-Rate-Limit-Count'])
        print(response.status_code)
        json_data = response.json()
        return json_data['accountId']
    #429 response means we hit the API rate limit 
    elif response.status_code == 429:
        print(response.status_code)
        print('Hit rate limit...Sleeping...')
        time.sleep(121)
        response = requests.get(url)
        json_data = response.json()
        return json_data['accountId']
    else:
        print(response.status_code)
        print('failed to get account id')
        return None

In [13]:
def get_match_history(acct_id, region, game_type, season,\
                      beg_ind, key):
    """
    returns past 100 match history as json
    use game_type '420' for 5v5 ranked
    season '11' for latest season
    """
    url = 'https://' + region + '.api.riotgames.com/lol/match/v4/matchlists/' +\
          'by-account/' + acct_id +'?queue=' + game_type + '&season=' + season +\
          '&beginIndex=' + str(beg_ind) + '&api_key=' + key
        
    response = requests.get(url)

    if response.status_code == 200:
        print('App Rate Limit ' + response.headers['X-App-Rate-Limit'])
        print('App Rate Limit Count ' + response.headers['X-App-Rate-Limit-Count'])
        print(response.status_code)
        json_data = response.json()
        return json_data
    #429 response means we hit the API rate limit 
    elif response.status_code == 429:
        print(response.status_code)
        print('Hit rate limit...Sleeping...')
        time.sleep(121)
        response = requests.get(url)
        json_data = response.json()
        return json_data
    else:
        print(response.status_code)
        return None

In [14]:
def get_match_history_int(acct_id, region, game_type, season, key, begin):
    """
    returns specified interval match history as json
    use game_type '420' for 5v5 ranked
    season '11' for latest season
    """
    url = 'https://' + region + '.api.riotgames.com/lol/match/v4/matchlists/' +\
          'by-account/' + acct_id +'?queue=' + game_type + '&season=' + season +\
          '&beginTime=' + str(begin) + '&api_key=' + key
    response = requests.get(url)
    
    if response.status_code == 200:
        print('App Rate Limit ' + response.headers['X-App-Rate-Limit'])
        print('App Rate Limit Count ' + response.headers['X-App-Rate-Limit-Count'])
        print(response.status_code)
        json_data = response.json()
        return json_data
    #429 response means we hit the API rate limit 
    elif response.status_code == 429:
        print(response.status_code)
        print('Hit rate limit...Sleeping...')
        time.sleep(121)
        response = requests.get(url)
        json_data = response.json()
        return json_data
    else:
        print(response.status_code)
        return None

In [15]:
def get_match_stats(match_id, region, key):
    """
    returns json of match stats based on match id
    """
    url = 'https://' + region +'.api.riotgames.com/lol/match/v4/matches/' +\
           str(match_id) + '?api_key=' + key
    response = requests.get(url)
    
    if response.status_code == 200:
        print('App Rate Limit ' + response.headers['X-App-Rate-Limit'])
        print('App Rate Limit Count ' + response.headers['X-App-Rate-Limit-Count'])
        print(response.status_code)
        json_data = response.json()
        return json_data
    #429 response means we hit the API rate limit 
    elif response.status_code == 429:
        print(response.status_code)
        print('Hit rate limit...Sleeping...')
        time.sleep(121)
        response = requests.get(url)
        json_data = response.json()
        return json_data
    else:
        print(response.status_code)
        print('No game data returned')
        return None

In [16]:
x = get_match_stats(2932547603, 'na1', api_key)

App Rate Limit 20:1,100:120
App Rate Limit Count 1:1,1:120
200


## Cleaning the Data

In [4]:
#Champion Information
champion_data_url = 'http://ddragon.leagueoflegends.com/cdn/9.20.1/data/en_US/champion.json'
champ_response = requests.get(champion_data_url)
champ_json = champ_response.json()
champ_json['data']

{'Aatrox': {'version': '9.20.1',
  'id': 'Aatrox',
  'key': '266',
  'name': 'Aatrox',
  'title': 'the Darkin Blade',
  'blurb': 'Once honored defenders of Shurima against the Void, Aatrox and his brethren would eventually become an even greater threat to Runeterra, and were defeated only by cunning mortal sorcery. But after centuries of imprisonment, Aatrox was the first to find...',
  'info': {'attack': 8, 'defense': 4, 'magic': 3, 'difficulty': 4},
  'image': {'full': 'Aatrox.png',
   'sprite': 'champion0.png',
   'group': 'champion',
   'x': 0,
   'y': 0,
   'w': 48,
   'h': 48},
  'tags': ['Fighter', 'Tank'],
  'partype': 'Blood Well',
  'stats': {'hp': 580,
   'hpperlevel': 90,
   'mp': 0,
   'mpperlevel': 0,
   'movespeed': 345,
   'armor': 38,
   'armorperlevel': 3.25,
   'spellblock': 32.1,
   'spellblockperlevel': 1.25,
   'attackrange': 175,
   'hpregen': 3,
   'hpregenperlevel': 1,
   'mpregen': 0,
   'mpregenperlevel': 0,
   'crit': 0,
   'critperlevel': 0,
   'attackdamag

In [5]:
champ_json['data']

{'Aatrox': {'version': '9.20.1',
  'id': 'Aatrox',
  'key': '266',
  'name': 'Aatrox',
  'title': 'the Darkin Blade',
  'blurb': 'Once honored defenders of Shurima against the Void, Aatrox and his brethren would eventually become an even greater threat to Runeterra, and were defeated only by cunning mortal sorcery. But after centuries of imprisonment, Aatrox was the first to find...',
  'info': {'attack': 8, 'defense': 4, 'magic': 3, 'difficulty': 4},
  'image': {'full': 'Aatrox.png',
   'sprite': 'champion0.png',
   'group': 'champion',
   'x': 0,
   'y': 0,
   'w': 48,
   'h': 48},
  'tags': ['Fighter', 'Tank'],
  'partype': 'Blood Well',
  'stats': {'hp': 580,
   'hpperlevel': 90,
   'mp': 0,
   'mpperlevel': 0,
   'movespeed': 345,
   'armor': 38,
   'armorperlevel': 3.25,
   'spellblock': 32.1,
   'spellblockperlevel': 1.25,
   'attackrange': 175,
   'hpregen': 3,
   'hpregenperlevel': 1,
   'mpregen': 0,
   'mpregenperlevel': 0,
   'crit': 0,
   'critperlevel': 0,
   'attackdamag

In [6]:
#Champ list
champ_list = []
for champ in champ_json['data']:
    champ_list.append(champ)


In [7]:
champ_list

['Aatrox',
 'Ahri',
 'Akali',
 'Alistar',
 'Amumu',
 'Anivia',
 'Annie',
 'Ashe',
 'AurelionSol',
 'Azir',
 'Bard',
 'Blitzcrank',
 'Brand',
 'Braum',
 'Caitlyn',
 'Camille',
 'Cassiopeia',
 'Chogath',
 'Corki',
 'Darius',
 'Diana',
 'Draven',
 'DrMundo',
 'Ekko',
 'Elise',
 'Evelynn',
 'Ezreal',
 'Fiddlesticks',
 'Fiora',
 'Fizz',
 'Galio',
 'Gangplank',
 'Garen',
 'Gnar',
 'Gragas',
 'Graves',
 'Hecarim',
 'Heimerdinger',
 'Illaoi',
 'Irelia',
 'Ivern',
 'Janna',
 'JarvanIV',
 'Jax',
 'Jayce',
 'Jhin',
 'Jinx',
 'Kaisa',
 'Kalista',
 'Karma',
 'Karthus',
 'Kassadin',
 'Katarina',
 'Kayle',
 'Kayn',
 'Kennen',
 'Khazix',
 'Kindred',
 'Kled',
 'KogMaw',
 'Leblanc',
 'LeeSin',
 'Leona',
 'Lissandra',
 'Lucian',
 'Lulu',
 'Lux',
 'Malphite',
 'Malzahar',
 'Maokai',
 'MasterYi',
 'MissFortune',
 'MonkeyKing',
 'Mordekaiser',
 'Morgana',
 'Nami',
 'Nasus',
 'Nautilus',
 'Neeko',
 'Nidalee',
 'Nocturne',
 'Nunu',
 'Olaf',
 'Orianna',
 'Ornn',
 'Pantheon',
 'Poppy',
 'Pyke',
 'Qiyana',
 'Qui

In [8]:
tag_list = []
tag_list2 = []
for i in champ_list:
    tag_list.append(champ_json['data'].get(i))
    
for x in tag_list:
    tag_list2.append(x.get('tags'))
tag_list2

[['Fighter', 'Tank'],
 ['Mage', 'Assassin'],
 ['Assassin'],
 ['Tank', 'Support'],
 ['Tank', 'Mage'],
 ['Mage', 'Support'],
 ['Mage'],
 ['Marksman', 'Support'],
 ['Mage'],
 ['Mage', 'Marksman'],
 ['Support', 'Mage'],
 ['Tank', 'Fighter'],
 ['Mage'],
 ['Support', 'Tank'],
 ['Marksman'],
 ['Fighter', 'Tank'],
 ['Mage'],
 ['Tank', 'Mage'],
 ['Marksman'],
 ['Fighter', 'Tank'],
 ['Fighter', 'Mage'],
 ['Marksman'],
 ['Fighter', 'Tank'],
 ['Assassin', 'Fighter'],
 ['Mage', 'Fighter'],
 ['Assassin', 'Mage'],
 ['Marksman', 'Mage'],
 ['Mage', 'Support'],
 ['Fighter', 'Assassin'],
 ['Assassin', 'Fighter'],
 ['Tank', 'Mage'],
 ['Fighter'],
 ['Fighter', 'Tank'],
 ['Fighter', 'Tank'],
 ['Fighter', 'Mage'],
 ['Marksman'],
 ['Fighter', 'Tank'],
 ['Mage', 'Support'],
 ['Fighter', 'Tank'],
 ['Fighter', 'Assassin'],
 ['Support', 'Mage'],
 ['Support', 'Mage'],
 ['Tank', 'Fighter'],
 ['Fighter', 'Assassin'],
 ['Fighter', 'Marksman'],
 ['Marksman', 'Mage'],
 ['Marksman'],
 ['Marksman'],
 ['Marksman'],
 ['Mag

In [9]:
tag_list3 = []
tag_list4 = []
for n in tag_list:
    tag_list3.append(n.get('info'))
for m in tag_list3:
    tag_list4.append(m.get('difficulty'))
tag_list4

[4,
 5,
 7,
 7,
 3,
 10,
 6,
 4,
 7,
 9,
 9,
 4,
 4,
 3,
 6,
 4,
 10,
 5,
 6,
 2,
 4,
 8,
 5,
 8,
 9,
 10,
 7,
 9,
 3,
 6,
 5,
 9,
 5,
 8,
 5,
 3,
 6,
 8,
 4,
 5,
 7,
 7,
 5,
 5,
 7,
 6,
 6,
 6,
 7,
 5,
 7,
 8,
 8,
 7,
 8,
 4,
 6,
 4,
 7,
 6,
 9,
 6,
 4,
 6,
 6,
 5,
 5,
 2,
 6,
 3,
 4,
 1,
 3,
 4,
 1,
 5,
 6,
 6,
 5,
 8,
 4,
 4,
 3,
 7,
 5,
 4,
 6,
 7,
 8,
 5,
 5,
 5,
 3,
 3,
 8,
 8,
 10,
 7,
 4,
 9,
 4,
 4,
 5,
 5,
 4,
 5,
 4,
 3,
 8,
 5,
 8,
 5,
 5,
 7,
 3,
 6,
 7,
 4,
 5,
 5,
 9,
 6,
 7,
 8,
 2,
 8,
 7,
 8,
 4,
 9,
 7,
 3,
 3,
 5,
 8,
 2,
 10,
 6,
 2,
 8,
 7,
 4,
 6,
 5,
 7]

In [10]:
champ_json['data']

{'Aatrox': {'version': '9.20.1',
  'id': 'Aatrox',
  'key': '266',
  'name': 'Aatrox',
  'title': 'the Darkin Blade',
  'blurb': 'Once honored defenders of Shurima against the Void, Aatrox and his brethren would eventually become an even greater threat to Runeterra, and were defeated only by cunning mortal sorcery. But after centuries of imprisonment, Aatrox was the first to find...',
  'info': {'attack': 8, 'defense': 4, 'magic': 3, 'difficulty': 4},
  'image': {'full': 'Aatrox.png',
   'sprite': 'champion0.png',
   'group': 'champion',
   'x': 0,
   'y': 0,
   'w': 48,
   'h': 48},
  'tags': ['Fighter', 'Tank'],
  'partype': 'Blood Well',
  'stats': {'hp': 580,
   'hpperlevel': 90,
   'mp': 0,
   'mpperlevel': 0,
   'movespeed': 345,
   'armor': 38,
   'armorperlevel': 3.25,
   'spellblock': 32.1,
   'spellblockperlevel': 1.25,
   'attackrange': 175,
   'hpregen': 3,
   'hpregenperlevel': 1,
   'mpregen': 0,
   'mpregenperlevel': 0,
   'crit': 0,
   'critperlevel': 0,
   'attackdamag

In [11]:
tag_list

[{'version': '9.20.1',
  'id': 'Aatrox',
  'key': '266',
  'name': 'Aatrox',
  'title': 'the Darkin Blade',
  'blurb': 'Once honored defenders of Shurima against the Void, Aatrox and his brethren would eventually become an even greater threat to Runeterra, and were defeated only by cunning mortal sorcery. But after centuries of imprisonment, Aatrox was the first to find...',
  'info': {'attack': 8, 'defense': 4, 'magic': 3, 'difficulty': 4},
  'image': {'full': 'Aatrox.png',
   'sprite': 'champion0.png',
   'group': 'champion',
   'x': 0,
   'y': 0,
   'w': 48,
   'h': 48},
  'tags': ['Fighter', 'Tank'],
  'partype': 'Blood Well',
  'stats': {'hp': 580,
   'hpperlevel': 90,
   'mp': 0,
   'mpperlevel': 0,
   'movespeed': 345,
   'armor': 38,
   'armorperlevel': 3.25,
   'spellblock': 32.1,
   'spellblockperlevel': 1.25,
   'attackrange': 175,
   'hpregen': 3,
   'hpregenperlevel': 1,
   'mpregen': 0,
   'mpregenperlevel': 0,
   'crit': 0,
   'critperlevel': 0,
   'attackdamage': 60,
  

In [12]:
df1 = pd.DataFrame(champ_list)
df1['tags'] = tag_list2
df1

0              tags
0     Aatrox   [Fighter, Tank]
1       Ahri  [Mage, Assassin]
2      Akali        [Assassin]
3    Alistar   [Tank, Support]
4      Amumu      [Tank, Mage]
..       ...               ...
140      Zed        [Assassin]
141    Ziggs            [Mage]
142   Zilean   [Support, Mage]
143      Zoe   [Mage, Support]
144     Zyra   [Mage, Support]

[145 rows x 2 columns]

In [13]:
df1.columns = ['Champion', 'tags']
df1

Champion              tags
0     Aatrox   [Fighter, Tank]
1       Ahri  [Mage, Assassin]
2      Akali        [Assassin]
3    Alistar   [Tank, Support]
4      Amumu      [Tank, Mage]
..       ...               ...
140      Zed        [Assassin]
141    Ziggs            [Mage]
142   Zilean   [Support, Mage]
143      Zoe   [Mage, Support]
144     Zyra   [Mage, Support]

[145 rows x 2 columns]

In [14]:
tag_list5 = []
for n in tag_list4:
    if n < 5:
        tag_list5.append('Easy')
    elif n < 8:
        tag_list5.append('Normal')
    elif n < 11:
        tag_list5.append('Hard')
tag_list5

['Easy',
 'Normal',
 'Normal',
 'Normal',
 'Easy',
 'Hard',
 'Normal',
 'Easy',
 'Normal',
 'Hard',
 'Hard',
 'Easy',
 'Easy',
 'Easy',
 'Normal',
 'Easy',
 'Hard',
 'Normal',
 'Normal',
 'Easy',
 'Easy',
 'Hard',
 'Normal',
 'Hard',
 'Hard',
 'Hard',
 'Normal',
 'Hard',
 'Easy',
 'Normal',
 'Normal',
 'Hard',
 'Normal',
 'Hard',
 'Normal',
 'Easy',
 'Normal',
 'Hard',
 'Easy',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Hard',
 'Hard',
 'Normal',
 'Hard',
 'Easy',
 'Normal',
 'Easy',
 'Normal',
 'Normal',
 'Hard',
 'Normal',
 'Easy',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Easy',
 'Normal',
 'Easy',
 'Easy',
 'Easy',
 'Easy',
 'Easy',
 'Easy',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Hard',
 'Easy',
 'Easy',
 'Easy',
 'Normal',
 'Normal',
 'Easy',
 'Normal',
 'Normal',
 'Hard',
 'Normal',
 'Normal',
 'Normal',
 'Easy',
 'Easy',
 'Hard',
 'Hard',
 'Hard',
 'Normal',
 'Easy',
 'Hard',
 '

In [15]:
df1['difficulty'] = tag_list5
df1

Champion              tags difficulty
0     Aatrox   [Fighter, Tank]       Easy
1       Ahri  [Mage, Assassin]     Normal
2      Akali        [Assassin]     Normal
3    Alistar   [Tank, Support]     Normal
4      Amumu      [Tank, Mage]       Easy
..       ...               ...        ...
140      Zed        [Assassin]     Normal
141    Ziggs            [Mage]       Easy
142   Zilean   [Support, Mage]     Normal
143      Zoe   [Mage, Support]     Normal
144     Zyra   [Mage, Support]     Normal

[145 rows x 3 columns]

In [16]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145 entries, 0 to 144
Data columns (total 3 columns):
Champion      145 non-null object
tags          145 non-null object
difficulty    145 non-null object
dtypes: object(3)
memory usage: 3.5+ KB


In [17]:
#Try to make a list of all genres
list1 = ['Fighter', 'Tank', 'Mage', 'Assassin', 'Support', 'Marksman']


In [20]:
list(df1.tags)

[['Fighter', 'Tank'],
 ['Mage', 'Assassin'],
 ['Assassin'],
 ['Tank', 'Support'],
 ['Tank', 'Mage'],
 ['Mage', 'Support'],
 ['Mage'],
 ['Marksman', 'Support'],
 ['Mage'],
 ['Mage', 'Marksman'],
 ['Support', 'Mage'],
 ['Tank', 'Fighter'],
 ['Mage'],
 ['Support', 'Tank'],
 ['Marksman'],
 ['Fighter', 'Tank'],
 ['Mage'],
 ['Tank', 'Mage'],
 ['Marksman'],
 ['Fighter', 'Tank'],
 ['Fighter', 'Mage'],
 ['Marksman'],
 ['Fighter', 'Tank'],
 ['Assassin', 'Fighter'],
 ['Mage', 'Fighter'],
 ['Assassin', 'Mage'],
 ['Marksman', 'Mage'],
 ['Mage', 'Support'],
 ['Fighter', 'Assassin'],
 ['Assassin', 'Fighter'],
 ['Tank', 'Mage'],
 ['Fighter'],
 ['Fighter', 'Tank'],
 ['Fighter', 'Tank'],
 ['Fighter', 'Mage'],
 ['Marksman'],
 ['Fighter', 'Tank'],
 ['Mage', 'Support'],
 ['Fighter', 'Tank'],
 ['Fighter', 'Assassin'],
 ['Support', 'Mage'],
 ['Support', 'Mage'],
 ['Tank', 'Fighter'],
 ['Fighter', 'Assassin'],
 ['Fighter', 'Marksman'],
 ['Marksman', 'Mage'],
 ['Marksman'],
 ['Marksman'],
 ['Marksman'],
 ['Mag

In [22]:
#first cham recommend
import random
val = input("Enter your feature: ['Fighter', 'Tank', 'Mage', 'Assassin', 'Support', 'Marksman'] ")  
list1 = []
count = 0
for i in list(df1.tags):
    if val in i:
        list1.append(count)
    count += 1
print(list1)   
print(df1.loc[random.choice(list1), 'Champion'])
        
    

Enter your feature: ['Fighter', 'Tank', 'Mage', 'Assassin', 'Support', 'Marksman'] Marksman
[7, 9, 14, 18, 21, 26, 35, 44, 45, 46, 47, 48, 55, 57, 59, 64, 71, 89, 104, 115, 117, 121, 124, 125, 133]
Sivir


In [27]:
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer

lista = []

mlb = MultiLabelBinarizer()

for column in df1:
    lista.append(mlb.fit_transform(df1[column]))
    print(lista)

cham_feature1 = pd.DataFrame(lista,columns=mlb.classes_, index=df1.index)
print(cham_feature1)


[array([[1, 0, 0, ..., 1, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 1, 0]])]
[array([[1, 0, 0, ..., 1, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 1, 0]]), array([[0, 1, 0, 0, 0, 1],
       [1, 0, 1, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 1],
       [0, 0, 1, 0, 0, 1],
       [0, 0, 1, 0, 1, 0],
       [0, 0, 1, 0, 0, 0],
       [0, 0, 0, 1, 1, 0],
       [0, 0, 1, 0, 0, 0],
       [0, 0, 1, 1, 0, 0],
       [0, 0, 1, 0, 1, 0],
       [0, 1, 0, 0, 0, 1],
       [0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 1, 1],
       [0, 0, 0, 1, 0, 0],
       [0, 1, 0, 0, 0, 1],
       [0, 0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0, 1],
       [0, 0, 0, 1, 0, 0],
       [0, 1, 0, 0, 0, 1],
       [0, 1, 1, 0, 0, 0],
       [0, 0, 0, 1,

ValueError: Shape of passed values is (3, 1), indices imply (145, 11)

In [28]:

import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer

df = pd.DataFrame(data = 
    {'groups':
        primary_list
    })

s = df['groups']

mlb = MultiLabelBinarizer()

cham_feature2 = pd.DataFrame(mlb.fit_transform(s),columns=mlb.classes_, index=df.index)
cham_feature2


NameError: name 'primary_list' is not defined

In [ ]:

import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer

df = pd.DataFrame(data = 
    {'groups':
        primary_list2
    })

s = df['groups']

mlb = MultiLabelBinarizer()

cham_feature3 = pd.DataFrame(mlb.fit_transform(s),columns=mlb.classes_, index=df.index)
cham_feature3


In [ ]:
mlb.classes_

In [ ]:
mlb = MultiLabelBinarizer()
s = df4['Primary']
mlb.fit(s)
mlb.classes_

In [ ]:
mlb.get_params

In [18]:
primary_list = []
for a in s:
    primary_list.append([a])
primary_list

NameError: name 's' is not defined

In [19]:
primary_list2 = []
for a in s2:
    primary_list2.append([a])
primary_list2

NameError: name 's2' is not defined

In [20]:
s2 = df4['Damage Type']
s2

NameError: name 'df4' is not defined

In [21]:
mlb.classes_

AttributeError: 'MultiLabelBinarizer' object has no attribute 'classes_'

In [22]:
mlb.fit_transform(df4['Damage'])

NameError: name 'df4' is not defined

In [23]:
df4 = df3.drop(columns = 'Champion')
df5 = df4.drop(columns = 'Primary')
df5 = df5.drop(columns = 'Secondary')
df5 = df5.drop(columns = 'Damage Type')
df5
cham_feature2
cham_feature3


NameError: name 'df3' is not defined

In [24]:
for column in df3:
    print(df3[column])


NameError: name 'df3' is not defined

In [206]:
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer

df = pd.DataFrame(data = 
    {'groups':
        df1.tags
    })

s = df['groups']

mlb = MultiLabelBinarizer()

cham_feature = pd.DataFrame(mlb.fit_transform(s),columns=mlb.classes_, index=df.index)
cham_feature
print(mlb.classes_)

['Assassin' 'Fighter' 'Mage' 'Marksman' 'Support' 'Tank']


In [207]:
s

0       [Fighter, Tank]
1      [Mage, Assassin]
2            [Assassin]
3       [Tank, Support]
4          [Tank, Mage]
             ...       
140          [Assassin]
141              [Mage]
142     [Support, Mage]
143     [Mage, Support]
144     [Mage, Support]
Name: groups, Length: 145, dtype: object

In [154]:
# import BeautifulSoup library
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait

In [155]:
from selenium import webdriver
import lxml.html as lh
import pandas as pd
import numpy as np

In [156]:
driver = webdriver.Chrome('/Users/GeluvcH/Desktop/LKL.PJ/Capstone_Project_1-master/chromedriver')

In [157]:
driver.get('https://leagueoflegends.fandom.com/wiki/List_of_champions/Ratings')

In [158]:
url='https://leagueoflegends.fandom.com/wiki/List_of_champions/Ratings'
#Create a handle, page, to handle the contents of the website
page = requests.get(url)
#Store the contents of the website under doc
doc = lh.fromstring(page.content)
#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

In [159]:
list001 = []
for i,e in enumerate(tr_elements):
    if i in range(10,155):
        #print(i)
        #print(e.text_content()+'-------------')
        #print(e.text_content())
        
        list001.append(e.text_content().split('\n'))
        
        #list001.append(''.join(e.text_content().split('\n')).split(' '))
#remove the useless'' in every element of the list   
list002=[]
for e in list001:
    temp_list=[]
    e.pop(12)
    e.pop(9)
    e.pop(0)
    for e1 in e:
        e1 = e1.replace(' \xa0', '');
        temp_list.append(e1)
    list002.append(temp_list)
   
print(list002)

[[' Aatrox', 'Juggernaut', '', ' 3', ' 3', ' 2', ' 2', ' 2', ' Physical', ' 2'], [' Ahri', 'Burst', '', ' 3', ' 1', ' 2', ' 3', ' 0', ' Magic', ' 2'], [' Akali', 'Assassin', '', ' 3', ' 1', ' 1', ' 3', ' 1', ' Mixed', ' 2'], [' Alistar', 'Vanguard', '', ' 1', ' 3', ' 3', ' 1', ' 2', ' Magic', ' 1'], [' Amumu', 'Vanguard', '', ' 2', ' 3', ' 3', ' 1', ' 0', ' Magic', ' 1'], [' Anivia', 'Battlemage', '', ' 3', ' 1', ' 3', ' 0', ' 2', ' Magic', ' 3'], [' Annie', 'Burst', '', ' 3', ' 1', ' 3', ' 1', ' 2', ' Magic', ' 1'], [' Ashe', 'Marksman', '', ' 2', ' 0', ' 3', ' 0', ' 2', ' Physical', ' 1'], [' Aurelion Sol', 'Battlemage', '', ' 3', ' 0', ' 2', ' 2', ' 0', ' Magic', ' 3'], [' Azir', 'Specialist', '', ' 3', ' 1', ' 2', ' 2', ' 1', ' Magic', ' 3'], [' Bard', 'Catcher', '', ' 1', ' 1', ' 3', ' 2', ' 3', ' Magic', ' 3'], [' Blitzcrank', 'Catcher', '', ' 1', ' 2', ' 3', ' 1', ' 0', ' Magic', ' 1'], [' Brand', 'Burst', '', ' 3', ' 0', ' 2', ' 0', ' 0', ' Magic', ' 2'], [' Braum', 'Warden', '

In [163]:
df3 = pd.DataFrame(np.array(list002),columns=['Champion', 'Primary', 'Secondary', 'Damage', 'Toughness', 'Control', 'Mobility', 'Utility', 'Damage Type', 'Difficulty'])
df3

Champion     Primary Secondary Damage Toughness Control Mobility Utility  \
0      Aatrox  Juggernaut                3         3       2        2       2   
1        Ahri       Burst                3         1       2        3       0   
2       Akali    Assassin                3         1       1        3       1   
3     Alistar    Vanguard                1         3       3        1       2   
4       Amumu    Vanguard                2         3       3        1       0   
..        ...         ...       ...    ...       ...     ...      ...     ...   
140       Zed    Assassin                3         0       1        3       0   
141     Ziggs   Artillery                3         0       2        2       0   
142    Zilean  Specialist                2         1       2        2       3   
143       Zoe       Burst                3         1       2        2       1   
144      Zyra     Catcher                3         0       3        0       1   

    Damage Type Difficulty  
0      Physical          2  
1         Magic          2  
2         Mixed          2  
3         Magic          1  
4         Magic          1  
..          ...        ...  
140    Physical          3  
141       Magic          2  
142       Magic          2  
143       Magic          3  
144       Magic          2  

[145 rows x 10 columns]

In [167]:
df3.drop(columns = 'Champion')

Primary Secondary Damage Toughness Control Mobility Utility  \
0    Juggernaut                3         3       2        2       2   
1         Burst                3         1       2        3       0   
2      Assassin                3         1       1        3       1   
3      Vanguard                1         3       3        1       2   
4      Vanguard                2         3       3        1       0   
..          ...       ...    ...       ...     ...      ...     ...   
140    Assassin                3         0       1        3       0   
141   Artillery                3         0       2        2       0   
142  Specialist                2         1       2        2       3   
143       Burst                3         1       2        2       1   
144     Catcher                3         0       3        0       1   

    Damage Type Difficulty  
0      Physical          2  
1         Magic          2  
2         Mixed          2  
3         Magic          1  
4         Magic          1  
..          ...        ...  
140    Physical          3  
141       Magic          2  
142       Magic          2  
143       Magic          3  
144       Magic          2  

[145 rows x 9 columns]

In [31]:
df

groups
0     [Fighter, Tank]
1    [Mage, Assassin]
2          [Assassin]
3     [Tank, Support]
4        [Tank, Mage]
..                ...
140        [Assassin]
141            [Mage]
142   [Support, Mage]
143   [Mage, Support]
144   [Mage, Support]

[145 rows x 1 columns]

In [350]:
df2 = pd.concat([cham_feature2, cham_feature3, df5, df1.Champion],axis=1,join='outer') #joinning table


In [351]:
df2

Artillery  Assassin  Battlemage  Burst  Catcher  Diver  Enchanter  \
0            0         0           0      0        0      0          0   
1            0         0           0      1        0      0          0   
2            0         1           0      0        0      0          0   
3            0         0           0      0        0      0          0   
4            0         0           0      0        0      0          0   
..         ...       ...         ...    ...      ...    ...        ...   
140          0         1           0      0        0      0          0   
141          1         0           0      0        0      0          0   
142          0         0           0      0        0      0          0   
143          0         0           0      1        0      0          0   
144          0         0           0      0        1      0          0   

     Juggernaut  Marksman  Skirmisher  ...   Magic   Mixed   Physical  Damage  \
0             1         0           0  ...       0       0          1       3   
1             0         0           0  ...       1       0          0       3   
2             0         0           0  ...       0       1          0       3   
3             0         0           0  ...       1       0          0       1   
4             0         0           0  ...       1       0          0       2   
..          ...       ...         ...  ...     ...     ...        ...     ...   
140           0         0           0  ...       0       0          1       3   
141           0         0           0  ...       1       0          0       3   
142           0         0           0  ...       1       0          0       2   
143           0         0           0  ...       1       0          0       3   
144           0         0           0  ...       1       0          0       3   

     Toughness  Control Mobility Utility Difficulty Champion  
0            3        2        2       2          2   Aatrox  
1            1        2        3       0          2     Ahri  
2            1        1        3       1          2    Akali  
3            3        3        1       2          1  Alistar  
4            3        3        1       0          1    Amumu  
..         ...      ...      ...     ...        ...      ...  
140          0        1        3       0          3      Zed  
141          0        2        2       0          2    Ziggs  
142          1        2        2       3          2   Zilean  
143          1        2        2       1          3      Zoe  
144          0        3        0       1          2     Zyra  

[145 rows x 23 columns]

In [352]:
#df2 = df2.rename(df1.Champion, axis='index')


In [300]:
df2.loc['Aatrox']

Artillery      0
Assassin       0
Battlemage     0
Burst          0
Catcher        0
Diver          0
Enchanter      0
Juggernaut     1
Marksman       0
Skirmisher     0
Specialist     0
Vanguard       0
Warden         0
 Magic         0
 Mixed         0
 Physical      1
Damage         3
Toughness      3
Control        2
Mobility       2
Utility        2
Difficulty     2
Name: Aatrox, dtype: object

In [353]:
df2 = df2.set_index('Champion') #set movie ID as index

In [354]:
df2

Artillery  Assassin  Battlemage  Burst  Catcher  Diver  Enchanter  \
Champion                                                                      
Aatrox            0         0           0      0        0      0          0   
Ahri              0         0           0      1        0      0          0   
Akali             0         1           0      0        0      0          0   
Alistar           0         0           0      0        0      0          0   
Amumu             0         0           0      0        0      0          0   
...             ...       ...         ...    ...      ...    ...        ...   
Zed               0         1           0      0        0      0          0   
Ziggs             1         0           0      0        0      0          0   
Zilean            0         0           0      0        0      0          0   
Zoe               0         0           0      1        0      0          0   
Zyra              0         0           0      0        1      0          0   

          Juggernaut  Marksman  Skirmisher  ...  Warden   Magic   Mixed  \
Champion                                    ...                           
Aatrox             1         0           0  ...       0       0       0   
Ahri               0         0           0  ...       0       1       0   
Akali              0         0           0  ...       0       0       1   
Alistar            0         0           0  ...       0       1       0   
Amumu              0         0           0  ...       0       1       0   
...              ...       ...         ...  ...     ...     ...     ...   
Zed                0         0           0  ...       0       0       0   
Ziggs              0         0           0  ...       0       1       0   
Zilean             0         0           0  ...       0       1       0   
Zoe                0         0           0  ...       0       1       0   
Zyra               0         0           0  ...       0       1       0   

           Physical  Damage  Toughness Control Mobility Utility Difficulty  
Champion                                                                    
Aatrox            1       3          3       2        2       2          2  
Ahri              0       3          1       2        3       0          2  
Akali             0       3          1       1        3       1          2  
Alistar           0       1          3       3        1       2          1  
Amumu             0       2          3       3        1       0          1  
...             ...     ...        ...     ...      ...     ...        ...  
Zed               1       3          0       1        3       0          3  
Ziggs             0       3          0       2        2       0          2  
Zilean            0       2          1       2        2       3          2  
Zoe               0       3          1       2        2       1          3  
Zyra              0       3          0       3        0       1          2  

[145 rows x 22 columns]

In [362]:
df2 = df2.astype('int64')


In [363]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 145 entries, Aatrox to Zyra
Data columns (total 22 columns):
Artillery     145 non-null int64
Assassin      145 non-null int64
Battlemage    145 non-null int64
Burst         145 non-null int64
Catcher       145 non-null int64
Diver         145 non-null int64
Enchanter     145 non-null int64
Juggernaut    145 non-null int64
Marksman      145 non-null int64
Skirmisher    145 non-null int64
Specialist    145 non-null int64
Vanguard      145 non-null int64
Warden        145 non-null int64
 Magic        145 non-null int64
 Mixed        145 non-null int64
 Physical     145 non-null int64
Damage        145 non-null int64
Toughness     145 non-null int64
Control       145 non-null int64
Mobility      145 non-null int64
Utility       145 non-null int64
Difficulty    145 non-null int64
dtypes: int64(22)
memory usage: 26.1+ KB


In [423]:
df51 = df5
            

In [424]:
df51 = df51.astype('int64')

In [432]:
df51['Difficulty'] = df51['Difficulty'].apply(lambda x: 0 if x == 0 or x == 1 else 1)

In [433]:
df51

Damage  Toughness  Control  Mobility  Utility  Difficulty
0         1          1        1         1        1           1
1         1          0        1         1        0           1
2         1          0        0         1        0           1
3         0          1        1         0        1           0
4         1          1        1         0        0           0
..      ...        ...      ...       ...      ...         ...
140       1          0        0         1        0           1
141       1          0        1         1        0           1
142       1          0        1         1        1           1
143       1          0        1         1        0           1
144       1          0        1         0        0           1

[145 rows x 6 columns]

In [435]:
df21 = df2

In [443]:
df21 = df21.drop(columns = ['Damage', 'Toughness', 'Control', 'Mobility', 'Utility', 'Difficulty'])

In [445]:
df21

Artillery  Assassin  Battlemage  Burst  Catcher  Diver  Enchanter  \
Champion                                                                      
Aatrox            0         0           0      0        0      0          0   
Ahri              0         0           0      1        0      0          0   
Akali             0         1           0      0        0      0          0   
Alistar           0         0           0      0        0      0          0   
Amumu             0         0           0      0        0      0          0   
...             ...       ...         ...    ...      ...    ...        ...   
Zed               0         1           0      0        0      0          0   
Ziggs             1         0           0      0        0      0          0   
Zilean            0         0           0      0        0      0          0   
Zoe               0         0           0      1        0      0          0   
Zyra              0         0           0      0        1      0          0   

          Juggernaut  Marksman  Skirmisher  Specialist  Vanguard  Warden  \
Champion                                                                   
Aatrox             1         0           0           0         0       0   
Ahri               0         0           0           0         0       0   
Akali              0         0           0           0         0       0   
Alistar            0         0           0           0         1       0   
Amumu              0         0           0           0         1       0   
...              ...       ...         ...         ...       ...     ...   
Zed                0         0           0           0         0       0   
Ziggs              0         0           0           0         0       0   
Zilean             0         0           0           1         0       0   
Zoe                0         0           0           0         0       0   
Zyra               0         0           0           0         0       0   

           Magic   Mixed   Physical  
Champion                             
Aatrox         0       0          1  
Ahri           1       0          0  
Akali          0       1          0  
Alistar        1       0          0  
Amumu          1       0          0  
...          ...     ...        ...  
Zed            0       0          1  
Ziggs          1       0          0  
Zilean         1       0          0  
Zoe            1       0          0  
Zyra           1       0          0  

[145 rows x 16 columns]

In [446]:
df51

Damage  Toughness  Control  Mobility  Utility  Difficulty
0         1          1        1         1        1           1
1         1          0        1         1        0           1
2         1          0        0         1        0           1
3         0          1        1         0        1           0
4         1          1        1         0        0           0
..      ...        ...      ...       ...      ...         ...
140       1          0        0         1        0           1
141       1          0        1         1        0           1
142       1          0        1         1        1           1
143       1          0        1         1        0           1
144       1          0        1         0        0           1

[145 rows x 6 columns]

In [478]:
df22 = pd.concat([cham_feature2, cham_feature3, df5, df1.Champion],axis=1,join='outer') #joinning table


In [479]:
df22 = df22.set_index('Champion') #set movie ID as index

In [746]:
df22

Artillery  Assassin  Battlemage  Burst  Catcher  Diver  Enchanter  \
Champion                                                                      
Aatrox            0         0           0      0        0      0          0   
Ahri              0         0           0      1        0      0          0   
Akali             0         1           0      0        0      0          0   
Alistar           0         0           0      0        0      0          0   
Amumu             0         0           0      0        0      0          0   
...             ...       ...         ...    ...      ...    ...        ...   
Zed               0         1           0      0        0      0          0   
Ziggs             1         0           0      0        0      0          0   
Zilean            0         0           0      0        0      0          0   
Zoe               0         0           0      1        0      0          0   
Zyra              0         0           0      0        1      0          0   

          Juggernaut  Marksman  Skirmisher  ...  Warden   Magic   Mixed  \
Champion                                    ...                           
Aatrox             1         0           0  ...       0       0       0   
Ahri               0         0           0  ...       0       1       0   
Akali              0         0           0  ...       0       0       1   
Alistar            0         0           0  ...       0       1       0   
Amumu              0         0           0  ...       0       1       0   
...              ...       ...         ...  ...     ...     ...     ...   
Zed                0         0           0  ...       0       0       0   
Ziggs              0         0           0  ...       0       1       0   
Zilean             0         0           0  ...       0       1       0   
Zoe                0         0           0  ...       0       1       0   
Zyra               0         0           0  ...       0       1       0   

           Physical  Damage  Toughness Control Mobility Utility Difficulty  
Champion                                                                    
Aatrox            1       3          3       2        2       2          2  
Ahri              0       3          1       2        3       0          2  
Akali             0       3          1       1        3       1          2  
Alistar           0       1          3       3        1       2          1  
Amumu             0       2          3       3        1       0          1  
...             ...     ...        ...     ...      ...     ...        ...  
Zed               1       3          0       1        3       0          3  
Ziggs             0       3          0       2        2       0          2  
Zilean            0       2          1       2        2       3          2  
Zoe               0       3          1       2        2       1          3  
Zyra              0       3          0       3        0       1          2  

[145 rows x 22 columns]

In [511]:
df11 = df1

In [512]:
df11.set_index('Champion')

tags difficulty
Champion                             
Aatrox     [Fighter, Tank]       Easy
Ahri      [Mage, Assassin]     Normal
Akali           [Assassin]     Normal
Alistar    [Tank, Support]     Normal
Amumu         [Tank, Mage]       Easy
...                    ...        ...
Zed             [Assassin]     Normal
Ziggs               [Mage]       Easy
Zilean     [Support, Mage]     Normal
Zoe        [Mage, Support]     Normal
Zyra       [Mage, Support]     Normal

[145 rows x 2 columns]

In [762]:
from scipy.spatial.distance import euclidean
from scipy.spatial.distance import hamming
# lets write a function to compute k nearest neighbours of active user


def comparisonFunc(chamname,champion_df,champion):
    index_list = []
    #print(list(champion_df['tags'][champion_df.Champion==champion])[0])
    chamname = chamname.set_index(pd.Index(range(len(chamname))))
    
    #print(chamname)
    for i,e in enumerate(chamname['tags']):
        #print(i,e)
        #print(e)
        for a in e:
            if a in list(champion_df['tags'][champion_df.Champion==champion])[0]:
                index_list.append(chamname.Champion[i])

    return list(set(index_list))

# To Find the k nearest neighbours of movies, first find the distance of input movie to all other movies.
def nearestneighbours(Champion, K, difficulty):
    # create a user df that contains all users except active user
    allChamName = pd.DataFrame(df22.index)
    allChamName = allChamName[allChamName.Champion!=Champion]
    # Add a column to this df which contains distance of active movie to each movie
    allChamName["distance"] = allChamName["Champion"].apply(lambda x: hamming(df22.loc[Champion],df22.loc[x]))
    allChamName["tags"] = df1.tags
    allChamName["difficulty"] = df1.difficulty
    #print(allChamName)
    #allChamName = allChamName[allChamName["tags"] in allChamName[allChamName["Champion"] == Champion]]
    # if difficulty is not none...
    allChamName = allChamName[allChamName.difficulty == difficulty]
    #print(allChamName)
    #print(list(df11['tags'][df11.Champion==Champion])[0])
    #print(allChamName)
    #print(comparisonFunc(allChamName,df11,Champion))
    allChamName = allChamName[allChamName['Champion'].isin(comparisonFunc(allChamName,df11,Champion))]
    #allChamName = allChamName[allChamName['Champion'] in comparisonFunc(allChamName,df11,Champion)]
    print(allChamName)
    KnearestUsers = allChamName.sort_values(["distance"],ascending=True)["Champion"][:K]
    KnearestUsersDistance = allChamName.sort_values(["distance"],ascending=True)["distance"][:K]
    #KnearestUsers = KnearestUsers[KnearestUser]
    return KnearestUsers,KnearestUsersDistance



In [763]:
nearestneighbours('TwistedFate', 10, 'Easy')

    Champion  distance              tags difficulty
4      Amumu  0.409091      [Tank, Mage]       Easy
12     Brand  0.272727            [Mage]       Easy
20     Diana  0.318182   [Fighter, Mage]       Easy
55    Kennen  0.318182  [Mage, Marksman]       Easy
69    Maokai  0.454545      [Tank, Mage]       Easy
74   Morgana  0.409091   [Mage, Support]       Easy
106     Sona  0.454545   [Support, Mage]       Easy
107   Soraka  0.409091   [Support, Mage]       Easy
124    Varus  0.045455  [Marksman, Mage]       Easy
138    Yuumi  0.409091   [Support, Mage]       Easy
141    Ziggs  0.227273            [Mage]       Easy


(124      Varus
 141      Ziggs
 12       Brand
 20       Diana
 55      Kennen
 4        Amumu
 74     Morgana
 107     Soraka
 138      Yuumi
 69      Maokai
 Name: Champion, dtype: object, 124    0.045455
 141    0.227273
 12     0.272727
 20     0.318182
 55     0.318182
 4      0.409091
 74     0.409091
 107    0.409091
 138    0.409091
 69     0.454545
 Name: distance, dtype: float64)

In [760]:
df1.at[124, 'tags'] = ['Marksman', 'Mage']

In [761]:
df1

Champion              tags difficulty
0     Aatrox   [Fighter, Tank]       Easy
1       Ahri  [Mage, Assassin]     Normal
2      Akali        [Assassin]     Normal
3    Alistar   [Tank, Support]     Normal
4      Amumu      [Tank, Mage]       Easy
..       ...               ...        ...
140      Zed        [Assassin]     Normal
141    Ziggs            [Mage]       Easy
142   Zilean   [Support, Mage]     Normal
143      Zoe   [Mage, Support]     Normal
144     Zyra   [Mage, Support]     Normal

[145 rows x 3 columns]

In [40]:
nearestneighbours('Annie', 3)

50      Karthus
127      Velkoz
130    Vladimir
Name: Champion, dtype: object

In [41]:
#Champ type
champ_type = {}

for champ in champ_list:
    champ_type[champ] = champ_json['data'][champ]['tags']

In [42]:
#Create dict to map champion ids to champ names
#will use to clean data later
champ_dict = {}

for champ in champ_list:
    champ_dict[int(champ_json['data'][champ]['key'])] = champ 

In [43]:
#Item Information
item_data_url = 'http://ddragon.leagueoflegends.com/cdn/8.24.1/data/en_US/item.json'
item_response = requests.get(item_data_url)
#print(item_response.status_code)
item_json = item_response.json()

In [44]:
#Access Item: item_json['data'][ITEM ID]['name']
item_json['data']['1001']['name']

#Item list
item_list = []
for item in item_json['data']:
    item_list.append(item)

In [45]:
#create a dict of item ids to map to item name for cleaning data
#Some of the item names were missing, went to api chat to find
item_dict = {}

for item in item_list:
    item_dict[int(item)] = item_json['data'][item]['name']

item_dict[0] = None

In [46]:
#Summoner Spell Info
sumspell_data_url = 'http://ddragon.leagueoflegends.com/cdn/8.24.1/data/en_US/summoner.json'
sumspell_response = requests.get(sumspell_data_url)
sumspell_json = sumspell_response.json()

In [47]:
#Access the sumspell ID num sumspell_json['data'][SUMSPELLNAME]['key']
#sumspell_json['data']['SummonerFlash']['key']

sumspell_list = []

for sumspell in sumspell_json['data']:
    sumspell_list.append(sumspell)

In [48]:
#create dict mapping of summoner spell id to name
sumspell_dict = {}
for spell in sumspell_list:
    sumspell_dict[int(sumspell_json['data'][spell]['key'])] = spell

#replace name of 'SummonerDot' to 'SummonerIgnite' like in game
sumspell_dict[14] = 'SummonerIgnite'

In [49]:
#testing 
#API key needs to be updated once expired
match_id = 2932547603
#return account_id
account_id = get_acct_id('Geluvch', 'na1', api_key)
print(account_id)
#get json data on past 100 matches
history = get_match_history(account_id, 'na1', '420', '11', 0, api_key)

App Rate Limit 20:1,100:120
App Rate Limit Count 1:1,1:120
200
DYA8qRwdrIHBsz_7UZeaOLciAl8Z9yScxg5aPjUxHUfUzQ
404


In [52]:
#df001 = pd.DataFrame(history['matches'])
# Exploring match history as a Pandas DataFrame
#print(df001.info())

## Building a Master Data Frame

In [21]:
#Lists of player names in first set
#Players are high tier ranked players from which we will draw match data from
players1 = ['scarra', 'gosu', 'trick2g' 'hashinshin', 'loltyler1', 'imaqtpie',
             'nightblue3', 'wingsofdeath', 'chapanya', 'shiphtur', 'doublelift',
            'voyboy', 'yassuo', 'gripex90','aphromoo', 'pokimane', 'dyrus', 'TF Blade',
           'pants are dragon', 'lourlo', 'allorim', 'Boxbôx', 'Per Se Bambi', 'horseie', 'Zile',
          'Nicolaj Jensen', 'pokeball', 'valkrin', 'hate', 'likeamaws', 'twinge',
          'sakurafloret', 'Chad jungle', 'mintcola', 'blobheart', 'joonroot', 'dominatelane',
           'vegeta', 'omnidk', 'dr beat', 'voki', 'kaipop', 'oogie', 'sophist sage',
           'takeji', 'zutter', 'haseeb', 'delyze', 'kicking', 'xiao ming',
           'SKT T1 Madlife', 'abibibi', 'Papá Chau', 'data88', 'wildtomgg',
           'bronson', 'd1 police', 'mercury','a noob draven', 'yao 17', 'xuegao',
           'treh', 'cdew', 'fakego', 'feng6', 'heisendong', 'pytrigon', 'billbobbilly', 'techxiii',
            'cao cao n1', 'honi', 'oddi3', 'adrian riv3n', 'xj11',
            'yu sang', 'meteos', 'ffwii', 'starwhale', 'iwdominatelol',
            'saskio', 'froogie', 'mancloud2', 'yoshi', 'haebaragi',
            'stuntopolis', 'wonhae', 'nopause', 'the holy slurp',
            '100T Bangg', '100t freeze', 'taproot86', 'bloodwater', 'catria', 'trinn',
            'nasfinest', 'im avi', 'liquid matt', 'dmytro', 'omizu', 'huhi',
            'TL cain', 'waynedwops', 'busioc', 'chaosrain', 'free papachau',
            'chris', '5fire', 'shiba inu', 'tsm bjergsen', '100t fragas', 'c9 sneaky',
           'thorn', 'deftsuo', 'adc abuser', 'dellzor', 'yisus', 'nebula', 'xin xin',
            'zoebestgurl', 'canadaad', 'rawvision', 'll stylish', 'oldhome', 'link',
            'rainbowboom', 'raigarak', 'thiengar', 'skeggsy', 'kyroo1', 'frosfroes',
            'wolfer', 'seouless', 'don tito', 'seanic', 'daddyio', 'aeriya',
            'quantum0002', 'ordinn', 'roja', 'boom u dead', 'rain lover',
            'jimmytheslayer', 'ikingvex', 'sun prince', 'morningst4r', 'zatsmod',
            'dragoonsmash', 'little mix', 'jim j poggers', 'll justice', 'shupian',
            'fwii', 'zaion', 'cile', 'gate', 'aesthetic player', 'duoqueen1',
            'urelectric', 'bigfatlp']

In [22]:
#Define function to get match history 
def get_many_matches(acct_names, key, file_name, add_on=False):
    '''
    Returns a list of match ids
    
    >>> l = [1,2,3,4]
    >>> with open("test.txt", "wb") as fp:   #Pickling
    ...   pickle.dump(l, fp)
    ... 
    >>> with open("test.txt", "rb") as fp:   # Unpickling
    ...   b = pickle.load(fp)
    ... 
    >>> b
    [1, 2, 3, 4]
    
    add_on = True -> add game ids to the specified pickled file
    add_on = False (default) -> create new pickled file of game ids
    '''
    #store matches in the list and keep count of api calls
    match_list = []
    bad_player_ids = []
    calls = 0
    
    #if the api is called 100 times, then sleep for 2 mins
    for player in acct_names:
        if calls == 100:
            print("API calls at 100, sleeping for 2 mins...")
            time.sleep(121)
            calls = 0
            
        acct_id = get_acct_id(player, 'na1', key)
        calls += 1
        
        if calls == 100:
            print("API calls at 100, sleeping for 2 mins...")
            time.sleep(121)
            calls = 0
        
        #Can sometimes return bad game data and return None
        #Handle the TypeError exception
        try:    
            beg_ind = 0
            data = get_match_history(acct_id, 'na1', '420', '11',\
                                     beg_ind, key)['matches']
            calls += 1
        except TypeError:
            #log current play and Skip current player and move on to the next
            bad_player_ids.append(player)
            continue
    
        while data != []:
            if calls == 100:
                print("API calls at 100, sleeping for 2 mins...")
                time.sleep(121)
                calls = 0
            
            data = get_match_history(acct_id, 'na1', '420', '11',\
                                     beg_ind, key)['matches']
            calls += 1
            
            for game in data:
                if game['gameId'] not in match_list:
                    match_list.append(game['gameId'])
            beg_ind += 100
            
    
    #store list using pickle for an existing file in 
    #the PlayerMatches directory
    if add_on == True:
        with open('PlayerMatches\\'+ file_name, 'rb') as file:
            old_list = pickle.load(file)
            old_list = list(set(old_list + match_list))
        with open('PlayerMatches\\'+ file_name, 'wb') as file:
            pickle.dump(old_list, file)
    
    #store list in a new file using pickle
    elif add_on == False:
        with open('PlayerMatches\\'+ file_name, 'wb') as file:
            pickle.dump(match_list, file)
    
    #print out player names for which we could not get match ids for
    if len(bad_player_ids) != 0:
        print('Could not return match ids for player: ' + str(bad_player_ids))
            
    

In [23]:
#add on to players1list.txt
#get_many_matches([], api_key, 'players1list.txt', add_on=True)

In [24]:
def create_row(json_data):
    #create all columns for dataframe
    row_contents = {}
    row_contents['game_id'] = json_data.get('gameId', 'unknown')
    row_contents['game_length_mins'] = json_data.get('gameDuration', 'unknown')
    add_player_name(row_contents, json_data)
    add_champ_pick(row_contents, json_data, champ_dict)
    add_sumspell_pick(row_contents, json_data, sumspell_dict)
    add_stats(row_contents, json_data, item_dict)
    add_team(row_contents, json_data)
    add_team_stats(row_contents, json_data)

    #return pd.DataFrame(row_contents, index=[0])
    return row_contents


### Defining some helper functions to clean data for create_row()

In [25]:
#Function to create player#:player name
def add_player_name(to_dict, json_data):
    for player in range(len(json_data.get('participantIdentities'))):
        to_dict['p'+str(player+1)+'_name'] = json_data['participantIdentities'][player]['player'].get('summonerName', 'unknown')
    print(json_data['gameId'])

In [26]:
#adds champion name
def add_champ_pick(to_dict, json_data, champ_map):
    for player in range(len(json_data['participantIdentities'])):
        champ = json_data['participants'][player].get('championId', 'unknown')
        to_dict['p'+str(player+1)+'_champ'] = champ
        if champ in champ_map:
            to_dict['p'+str(player+1)+'_champ'] = champ_map[to_dict['p'+str(player+1)+'_champ']]

In [27]:
#addes picks of summoner spells
def add_sumspell_pick(to_dict, json_data, spell_map):
    for player in range(len(json_data['participantIdentities'])):
        spell1 = json_data['participants'][player].get('spell1Id', 'unknown')
        spell2 = json_data['participants'][player].get('spell2Id', 'unknown')
        to_dict['p'+str(player+1)+'_spell1'] = spell1
        to_dict['p'+str(player+1)+'_spell2'] = spell2
        if spell1 in sumspell_dict:
            to_dict['p'+str(player+1)+'_spell1'] = sumspell_dict[to_dict['p'+str(player+1)+'_spell1']]
        if spell2 in sumspell_dict:
            to_dict['p'+str(player+1)+'_spell2'] = sumspell_dict[to_dict['p'+str(player+1)+'_spell2']]

In [28]:
#define function to add stats to Data Frame
def add_stats(to_dict, json_data, item_map):
    pattern = re.compile('item\d')
    for player in range(len(json_data['participants'])):
        for stat in json_data['participants'][player]['stats']:
            stat_name = json_data['participants'][player]['stats'].get(stat, 'unknown')
            to_dict['p'+str(player+1)+stat] = stat_name
            #if item not in item_map it means its not a current active item as of patch
            if (bool(pattern.match(stat)) == True) and (stat_name in item_map):
                to_dict['p'+str(player+1)+stat] = item_map[to_dict['p'+str(player+1)+stat]]

In [29]:
#determine team
def add_team(to_dict, json_data):
    for player in range(len(json_data.get('participantIdentities'))):
        to_dict['p'+str(player+1)+'_team'] = json_data['participants'][player].get('teamId', 'unknown')
        if to_dict['p'+str(player+1)+'_team'] == 100:
            to_dict['p'+str(player+1)+'_team'] = 'blue'
       
    else:
            to_dict['p'+str(player+1)+'_team'] = 'red'

In [30]:
#add stats from all 10 players
def add_team_stats(to_dict, json_data):
    for team in range(len(json_data['teams'])):
        if json_data['teams'][team]['teamId'] == 100:
            for stat in json_data['teams'][team]:
                to_dict['blue_team_'+ stat] = json_data['teams'][team].get(stat, 'unknown')
            del to_dict['blue_team_bans']
            del to_dict['blue_team_teamId']
        else:
            for stat in json_data['teams'][team]:
                to_dict['red_team_'+ stat] = json_data['teams'][team].get(stat, 'unknown')
            del to_dict['red_team_bans']
            del to_dict['red_team_teamId']


### Putting it all together

In [36]:
def create_master_data(match_list, region, key, out_name=None, csv=False):
    #Create csv files of about 10,000 games at a time
    '''
    If csv=True, then the function will create a .csv of the match data,
    otherwise return a dataframe
    
    match_list will be the pickled list of match ids using get_many_matches()
    in PlayerMatches directory
    match_list: pickled filed of match ids in PlayerMatches directory
    '''
    match_data = []
    failed_games = []
    loop_count = 0
    
    
    with open('PlayerMatches/'+ match_list, 'rb') as file:
        matches = pickle.load(file)
        #Uncomment below to test for 200 matches
        #matches = matches[:200]
  
    #Limit this loop to 100 calls every 2 mins   
    for match in matches:
        try:
            if loop_count == 100:
                print("wait 2 minutes before next call")
                time.sleep(121)
                loop_count = 0

            loop_count += 1
            stats = get_match_stats(match, region, key)

            if stats == None:
                print('Skipping Game...')
                continue

            row = create_row(stats)
            match_data.append(row)
        except Exception as e:
            print(e)
            print('Failed to get data for game: ' + str(match))
            failed_games.append(match)
            continue
            
        
    if csv == False:
        return pd.DataFrame(match_data)

    elif csv == True:
        out_file = 'PlayerMatches/' + str(out_name) + '.csv'
        x = pd.DataFrame(match_data)
        x.to_csv(out_file)
        
        with open('PlayerMatches/'+ 'failedgames.txt', 'wb') as file:
            pickle.dump(failed_games, file)
        
    #handle exception if there is an error and write current data to file
    #and print out last match id so I can start back from there

In [ ]:
create_master_data('matchid_3.txt', 'na1', api_key, out_name='match_data_3', csv=True)

App Rate Limit 20:1,100:120
App Rate Limit Count 1:1,1:120
200
2733608895
App Rate Limit 20:1,100:120
App Rate Limit Count 2:1,2:120
200
2733594506
App Rate Limit 20:1,100:120
App Rate Limit Count 1:1,3:120
200
2733116639
App Rate Limit 20:1,100:120
App Rate Limit Count 2:1,4:120
200
2733124225
App Rate Limit 20:1,100:120
App Rate Limit Count 1:1,5:120
200
2733111615
App Rate Limit 20:1,100:120
App Rate Limit Count 2:1,6:120
200
2733079078
App Rate Limit 20:1,100:120
App Rate Limit Count 1:1,7:120
200
2732978321
App Rate Limit 20:1,100:120
App Rate Limit Count 2:1,8:120
200
2732938350
App Rate Limit 20:1,100:120
App Rate Limit Count 1:1,9:120
200
2732922575
App Rate Limit 20:1,100:120
App Rate Limit Count 2:1,10:120
200
2732894764
App Rate Limit 20:1,100:120
App Rate Limit Count 1:1,11:120
200
2732155912
App Rate Limit 20:1,100:120
App Rate Limit Count 2:1,12:120
200
2732126580
App Rate Limit 20:1,100:120
App Rate Limit Count 1:1,13:120
200
2732098390
App Rate Limit 20:1,100:120
App Ra

App Rate Limit 20:1,100:120
App Rate Limit Count 1:1,11:120
200
2719066187
App Rate Limit 20:1,100:120
App Rate Limit Count 2:1,12:120
200
2719050783
App Rate Limit 20:1,100:120
App Rate Limit Count 1:1,13:120
200
2718638730
App Rate Limit 20:1,100:120
App Rate Limit Count 2:1,14:120
200
2718592205
App Rate Limit 20:1,100:120
App Rate Limit Count 1:1,15:120
200
2718570840
App Rate Limit 20:1,100:120
App Rate Limit Count 2:1,16:120
200
2718547600
App Rate Limit 20:1,100:120
App Rate Limit Count 1:1,17:120
200
2718553638
App Rate Limit 20:1,100:120
App Rate Limit Count 2:1,18:120
200
2718519091
App Rate Limit 20:1,100:120
App Rate Limit Count 1:1,19:120
200
2718530079
App Rate Limit 20:1,100:120
App Rate Limit Count 2:1,20:120
200
2717616413
App Rate Limit 20:1,100:120
App Rate Limit Count 1:1,21:120
200
2717567827
App Rate Limit 20:1,100:120
App Rate Limit Count 2:1,22:120
200
2717541073
App Rate Limit 20:1,100:120
App Rate Limit Count 1:1,23:120
200
2717487204
App Rate Limit 20:1,100:1

App Rate Limit 20:1,100:120
App Rate Limit Count 2:1,20:120
200
2700551501
App Rate Limit 20:1,100:120
App Rate Limit Count 1:1,21:120
200
2699475444
App Rate Limit 20:1,100:120
App Rate Limit Count 2:1,22:120
200
2699482429
App Rate Limit 20:1,100:120
App Rate Limit Count 1:1,23:120
200
2699469891
App Rate Limit 20:1,100:120
App Rate Limit Count 2:1,24:120
200
2698406256
App Rate Limit 20:1,100:120
App Rate Limit Count 1:1,25:120
200
2697666911
App Rate Limit 20:1,100:120
App Rate Limit Count 1:1,26:120
200
2697659514
App Rate Limit 20:1,100:120
App Rate Limit Count 2:1,27:120
200
2697656158
App Rate Limit 20:1,100:120
App Rate Limit Count 1:1,28:120
200
2697631529
App Rate Limit 20:1,100:120
App Rate Limit Count 2:1,29:120
200
2697611698
App Rate Limit 20:1,100:120
App Rate Limit Count 1:1,30:120
200
2697584090
App Rate Limit 20:1,100:120
App Rate Limit Count 2:1,31:120
200
2697556466
App Rate Limit 20:1,100:120
App Rate Limit Count 1:1,32:120
200
2696738302
App Rate Limit 20:1,100:1

In [33]:
pwd

'/Users/GeluvcH/Desktop/LKL.PJ/Capstone_Project_1-master'

In [ ]:
#save list of ids 10,000 at a time using naming convention matchid_#
#from the total list of ids make sublists of 10,000 ids for create_master_data() 

#DO THIS BY:
    
#with open("PlayerMatches\\players1list.txt", "rb") as file:
#    big_list = pickle.load(file)

#FIRST 10000 games
#with open("PlayerMatches\\matchid_1.txt", "wb") as file:
#    sublist = big_list[:10000] 
#    pickle.dump(sublist, file)

#NEXT 10000 games
#with open("PlayerMatches\\matchid_5.txt", "wb") as file:
#    sublist = big_list[40000:50000] 
#    pickle.dump(sublist, file)